<a href="https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/transition-matrix-gee-glance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install geemap -q

In [41]:
# Import libraries
import pandas as pd
import ee
import os
import sys
import geemap


# Define repository details
repo_url = 'https://github.com/antoniovfonseca/agentic-ai-global-lulc.git'
repo_name = 'agentic-ai-global-lulc'

# Clone or update the repository
if not os.path.exists(repo_name):
    print(f"Cloning {repo_name}...")
    !git clone {repo_url}
else:
    print(f"Updating {repo_name}...")
    !cd {repo_name} && git pull

# Add the notebooks directory to system path to allow imports
notebooks_path = os.path.join(repo_name, 'notebooks')
if notebooks_path not in sys.path:
    sys.path.append(os.path.abspath(notebooks_path))
    print(f"Added {notebooks_path} to system path.")

import importlib
import utils

importlib.reload(utils)
print("Utils module loaded successfully.")

Updating agentic-ai-global-lulc...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 381 bytes | 190.00 KiB/s, done.
From https://github.com/antoniovfonseca/agentic-ai-global-lulc
   108ed15..d28687d  main       -> origin/main
Updating 108ed15..d28687d
Fast-forward
 notebooks/utils.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Added agentic-ai-global-lulc/notebooks to system path.
Utils module loaded successfully.


In [24]:
import ee
import geemap
import pandas as pd
import utils  # Certifique-se que o utils.py está na mesma pasta ou no sys.path

# Autenticação e Inicialização
try:
    ee.Initialize(project='ee-antoniovicgf')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-antoniovicgf')

In [22]:
# 1. Define the full temporal range from 2001 to 2019
full_year_list = list(range(2001, 2020))

# 2. Execute the batch calculation for all transitions
# This returns a dictionary with 18 annual matrices and 1 long-term matrix
all_transition_matrices = utils.calculate_global_transition_matrix(
    year_list=full_year_list,
    scale=300
)

# 3. Example: Display the long-term transition (2001 to 2019)
print("Long-term transition matrix (2001-2019):")
display(all_transition_matrices["2001_2019"])

# 4. Example: Access a specific annual change (e.g., 2005 to 2006)
# display(all_transition_matrices["2005_2006"])

Successfully computed: 2001_2002
Successfully computed: 2002_2003
Successfully computed: 2003_2004
Successfully computed: 2004_2005
Successfully computed: 2005_2006
Successfully computed: 2006_2007


Successfully computed: 2007_2008


KeyboardInterrupt: 

In [25]:
# 1. Define the full range of years (2001 to 2019)
# This will result in 18 annual transitions + 1 long-term transition
years_to_process = list(range(2001, 2020))

# 2. Trigger the export tasks via the updated utils function
# Scale=30 ensures native resolution; drive_folder is where CSVs will appear
tasks_list = utils.export_global_transition_tasks(
    year_list=years_to_process,
    drive_folder="GLANCE_Global_Transitions_30m",
    scale=30
)

# 3. Quick overview of the triggered tasks
print(f"\nSuccessfully triggered {len(tasks_list)} export tasks.")
print("You can monitor progress in the GEE Tasks tab or via code.")

Task started for transition_2001_2002 (Scale: 30m)
Task started for transition_2002_2003 (Scale: 30m)
Task started for transition_2003_2004 (Scale: 30m)
Task started for transition_2004_2005 (Scale: 30m)
Task started for transition_2005_2006 (Scale: 30m)
Task started for transition_2006_2007 (Scale: 30m)
Task started for transition_2007_2008 (Scale: 30m)
Task started for transition_2008_2009 (Scale: 30m)
Task started for transition_2009_2010 (Scale: 30m)
Task started for transition_2010_2011 (Scale: 30m)
Task started for transition_2011_2012 (Scale: 30m)
Task started for transition_2012_2013 (Scale: 30m)
Task started for transition_2013_2014 (Scale: 30m)
Task started for transition_2014_2015 (Scale: 30m)
Task started for transition_2015_2016 (Scale: 30m)
Task started for transition_2016_2017 (Scale: 30m)
Task started for transition_2017_2018 (Scale: 30m)
Task started for transition_2018_2019 (Scale: 30m)
Task started for transition_2001_2019 (Scale: 30m)

Successfully triggered 19 expo

In [31]:
# Optional cell to monitor the status of the tasks
for task in tasks_list:
    status = task.status()
    print(f"Task: {status['description']} | Status: {status['state']}")

Task: transition_2001_2002 | Status: COMPLETED
Task: transition_2002_2003 | Status: COMPLETED
Task: transition_2003_2004 | Status: COMPLETED
Task: transition_2004_2005 | Status: COMPLETED
Task: transition_2005_2006 | Status: COMPLETED
Task: transition_2006_2007 | Status: COMPLETED
Task: transition_2007_2008 | Status: COMPLETED
Task: transition_2008_2009 | Status: COMPLETED
Task: transition_2009_2010 | Status: COMPLETED
Task: transition_2010_2011 | Status: FAILED
Task: transition_2011_2012 | Status: COMPLETED
Task: transition_2012_2013 | Status: COMPLETED
Task: transition_2013_2014 | Status: COMPLETED
Task: transition_2014_2015 | Status: COMPLETED
Task: transition_2015_2016 | Status: COMPLETED
Task: transition_2016_2017 | Status: COMPLETED
Task: transition_2017_2018 | Status: COMPLETED
Task: transition_2018_2019 | Status: COMPLETED
Task: transition_2001_2019 | Status: COMPLETED


In [28]:
import time
from tqdm.notebook import tqdm

# 1. Initialize the progress bar with the total number of tasks
pbar = tqdm(total=len(tasks_list), desc="Exporting GLANCE Matrices")

# 2. Track which tasks have already been counted as finished
completed_task_ids = set()

# 3. Monitor loop: runs until all tasks are COMPLETED or FAILED
while len(completed_task_ids) < len(tasks_list):
    for task in tasks_list:
        # 4. Skip if the task was already processed in previous iterations
        if task.id in completed_task_ids:
            continue

        # 5. Check current status from GEE server
        status = task.status()
        state = status['state']

        # 6. Update progress bar if task reaches a terminal state
        if state in ['COMPLETED', 'FAILED', 'CANCELLED']:
            completed_task_ids.add(task.id)
            pbar.update(1)

            if state == 'FAILED':
                print(f"\nTask {status['description']} failed: {status.get('error_message')}")

    # 7. Wait 30 seconds before polling the server again to avoid overhead
    if len(completed_task_ids) < len(tasks_list):
        time.sleep(30)

pbar.close()
print("All export tasks have been processed.")

Exporting GLANCE Matrices:   0%|          | 0/19 [00:00<?, ?it/s]


Task transition_2010_2011 failed: Computation timed out.


All export tasks have been processed.


In [36]:
# 1. Mount Google Drive to access the files
from google.colab import drive
drive.mount('/content/drive')

# 2. Define the path where the CSVs were saved
# Adjust this path according to your Drive structure
path_to_csvs = "/content/drive/MyDrive/GLANCE_Global_Transitions_30m"

# 3. Load and convert all files into transition matrices
matrices_dict = utils.load_global_transition_matrices(
    drive_path=path_to_csvs
)

# 4. Access any specific matrix (e.g., the long-term one)
display(matrices_dict["2001_2019"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To,Barren,Developed,Herbaceous,Ice/Snow,Shrub,Trees,Water
From,,,,,,,
Barren,4617781024,1372738,143825256,66309,225385078,19187688,11943288
Developed,1273519,878443767,16370403,1238,2556607,2987554,355686
Herbaceous,117850415,43780963,36671595178,29008,636418755,1313266252,65037940
Ice/Snow,126481,484,19992,6548068404,2559,59447,27013
Shrub,54030152,2171602,889707143,14663,8488427463,469446939,2961913
Trees,13485698,16820397,1731886691,56686,507533337,33404868559,20693272
Water,32701720,646883,71946482,39158,3352176,15062043,18760485145


In [38]:
# 1. Convert pixel matrices to area matrices (km2)
# Using the dictionary 'matrices_dict' loaded in the previous step
area_results = utils.convert_matrices_to_area(
    matrices_dict=matrices_dict,
    pixel_size=30
)

# 2. Display the long-term transition in km2
print("Global Transition Matrix 2001-2019 (Values in km²):")
display(area_results["2001_2019"])

Global Transition Matrix 2001-2019 (Values in km²):


To,Barren,Developed,Herbaceous,Ice/Snow,Shrub,Trees,Water
From,,,,,,,
Barren,4156002.92,1235.46,129442.73,59.68,202846.57,17268.92,10748.96
Developed,1146.17,790599.39,14733.36,1.11,2300.95,2688.80,320.12
Herbaceous,106065.37,39402.87,33004435.66,26.11,572776.88,1181939.63,58534.15
Ice/Snow,113.83,0.44,17.99,5893261.56,2.30,53.50,24.31
Shrub,48627.14,1954.44,800736.43,13.20,7639584.72,422502.25,2665.72
Trees,12137.13,15138.36,1558698.02,51.02,456780.00,30064381.70,18623.94
Water,29431.55,582.19,64751.83,35.24,3016.96,13555.84,16884436.63


In [42]:
# 1. Define the destination folder in your mounted Google Drive
# Replace this path with your preferred Drive directory
drive_output_path = "/content/drive/MyDrive/GLANCE_Global_Transitions_30m/results_km2"

# 2. Call the utility function to export all matrices
# This will save all 19 matrices (18 annual + 1 long-term)
exported_files = utils.save_area_matrices_to_csv(
    area_matrices=area_results,
    output_dir=drive_output_path
)

# 3. Final summary of the operation
print(f"\nOperation complete: {len(exported_files)} matrices are now in your Drive.")

Successfully saved: transition_matrix_km2_2002_2003.csv
Successfully saved: transition_matrix_km2_2001_2002.csv
Successfully saved: transition_matrix_km2_2003_2004.csv
Successfully saved: transition_matrix_km2_2004_2005.csv
Successfully saved: transition_matrix_km2_2005_2006.csv
Successfully saved: transition_matrix_km2_2006_2007.csv
Successfully saved: transition_matrix_km2_2007_2008.csv
Successfully saved: transition_matrix_km2_2008_2009.csv
Successfully saved: transition_matrix_km2_2009_2010.csv
Successfully saved: transition_matrix_km2_2011_2012.csv
Successfully saved: transition_matrix_km2_2012_2013.csv
Successfully saved: transition_matrix_km2_2013_2014.csv
Successfully saved: transition_matrix_km2_2014_2015.csv
Successfully saved: transition_matrix_km2_2015_2016.csv
Successfully saved: transition_matrix_km2_2016_2017.csv
Successfully saved: transition_matrix_km2_2017_2018.csv
Successfully saved: transition_matrix_km2_2018_2019.csv
Successfully saved: transition_matrix_km2_2001_2